# Analyze results
This notebook takes the outcomes of `1. Tree filter.ipynb`, `3. Extract tree shapes.ipynb` and `4. Extract tree trunks.ipynb`. It compares these predictions against ground thruth data and presents some statistics and visualizations. 

In [ ]:
import pandas as pd
import geopandas as gpd
from shapely import wkt

import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import contextily as cx

import pathlib

In [ ]:
DATA_FOLDER = pathlib.Path('../data')
DATA_DIR = "/home/aiteam/projects/BOA/Tree_Detection_in_Aerial_Point_Clouds/datasets/" #TODO remove

# Input paths
measure_file = DATA_FOLDER / "ground_truth/gt_trees.gpkg"
area_file = DATA_FOLDER / "ground_truth/gt_area.gpkg"

ahn_results_file = DATA_DIR + "HGB/Trunks/Output_centroids_only_124_486_125_486.shp" # TODO adjust
ahn_results_area_file = DATA_FOLDER / "shapes/trees.csv"

# Output paths
output_data = DATA_FOLDER / "final_results/final_analysis_data.gpkg"
output_image = DATA_FOLDER / "final_results/final_image.png"
output_image_analysis = DATA_FOLDER / "final_results/final_analysis_image.png"

In [ ]:
CRS = "epsg:28992"

# Maximum distance to be identified as the same tree (m)
max_dist = 0.7

## Import data

### Area

In [ ]:
df_area = gpd.read_file(area_file)

### Measurements

In [ ]:
gdf = gpd.read_file(measure_file)

### AHN results

#### trunks

In [ ]:
df_ahn = gpd.read_file(ahn_results_file)
df_ahn = df_ahn.set_crs(CRS, allow_override=True)

#### shapes

In [ ]:
use_concave = True
df_ahn_areas = gpd.read_file(ahn_results_area_file, crs=CRS)
df_ahn_areas["location"] = df_ahn_areas["location"].apply(wkt.loads)
df_ahn_areas["convex_hull"] = df_ahn_areas["convex_hull"].apply(wkt.loads)
if use_concave:
    df_ahn_areas["concave_hull"] = df_ahn_areas["concave_hull"].apply(wkt.loads)
    df_ahn_areas.set_geometry("concave_hull", inplace=True)
else:
    df_ahn_areas.set_geometry("convex_hull", inplace=True)
df_ahn_areas.drop("geometry", axis=1, inplace=True)
df_ahn_areas = df_ahn_areas.set_crs(CRS)
df_ahn_areas.drop_duplicates(["convex_hull", "concave_hull"], inplace=True)

### Pre-process data

In [ ]:
# Select trees that are within the maintainance area
gdf_sel = gdf.sjoin(df_area[["geometry"]], predicate="within").drop(
    ["index_right"], axis=1
)
df_ahn_sel = df_ahn.sjoin(df_area[["geometry"]], predicate="within").drop(
    ["index_right"], axis=1
)

df_ahn_areas_sel = gpd.clip(df_ahn_areas, df_area)
df_ahn_areas_sel_dis = gpd.GeoDataFrame(
    geometry=gpd.GeoSeries(df_ahn_areas_sel.unary_union.geoms)
)

## Get results

### Find matches

In [ ]:
gdf_sel['objectid'] = gdf_sel.index

In [ ]:
# Get nearest measured tree to predicted trees
df_ahn_sjoin = df_ahn_sel.sjoin_nearest(
    gdf_sel[["objectid", "geometry"]], distance_col="distance", how="left"
)

In [ ]:
# Only keep cases where predicted trees are also closest to measured trees
df_ahn_sjoin.sort_values(["objectid", "distance"], inplace=True)
df_ahn_sjoin.drop_duplicates(subset=["objectid"], keep="first", inplace=True)

# Drop cases where the distance between measured and predicated tree is too large
df_ahn_sjoin = df_ahn_sjoin[df_ahn_sjoin["distance"] < max_dist]

### Calculate overall statistics

In [ ]:
true_positives = gdf_sel[
    gdf_sel["objectid"].isin(df_ahn_sjoin["objectid"])
].reset_index(drop=True)
false_negatives = gdf_sel[
    ~gdf_sel["objectid"].isin(true_positives["objectid"])
].reset_index(drop=True)
false_positives = df_ahn_sel[
    ~df_ahn_sel["label"].isin(df_ahn_sjoin["label"])
].reset_index(drop=True)

In [ ]:
print("measured: " + str(len(gdf_sel)))
print("predicted: " + str(len(df_ahn_sel)))
print("TP: " + str(len(true_positives)))
print("FN: " + str(len(false_negatives)))
print("FP: " + str(len(false_positives)))
print(
    "precision: "
    + str(round(len(true_positives) / (len(true_positives) + len(false_positives)), 2))
)
print(
    "recall: "
    + str(round(len(true_positives) / (len(true_positives) + len(false_negatives)), 2))
)

### Store

In [ ]:
true_positives['type'] = 'TP'
false_negatives['type'] = 'FN'
false_positives['type'] = 'FP'
df_final = pd.concat([true_positives[['geometry', 'type']],
                     false_negatives[['geometry', 'type']],
                     false_positives[['geometry', 'type']]])

In [ ]:
df_final.to_file(output_data, driver="GPKG")

## Plot results

In [ ]:
fig, ax = plt.subplots(figsize=(12, 12), frameon=False, dpi=500)

# Area
df_area.boundary.plot(ax=ax, color="blue")

# AHN results - areas
df_ahn_areas_sel_dis.plot(ax=ax, color="purple", alpha=0.3)

# Measurements
gdf_sel.plot(ax=ax, color="yellow", alpha=0.2, markersize=16)

# AHN results - trunks
df_ahn_sel.plot(ax=ax, color="purple", alpha=0.7, markersize=6)

# Background
cx.add_basemap(ax=ax, source=cx.providers.Esri.WorldImagery, crs=CRS)

ax.axis("off")

# Create legend
ar = mpatches.Patch(facecolor="silver", edgecolor="blue", label="area")
me = mpatches.Patch(color="yellow", alpha=0.5, label="measurements")
pr1 = mpatches.Patch(color="purple", label="predictions (AHN) - trunks")
pr2 = mpatches.Patch(color="purple", alpha=0.3, label="predictions (AHN) - trees")
plt.legend(handles=[ar, me, pr1, pr2], loc="lower right")

plt.savefig(output_image, bbox_inches="tight")
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(12, 12), frameon=False, dpi=500)

# Area
df_area.boundary.plot(ax=ax, color="blue")

# AHN results - areas
df_ahn_areas_sel_dis.plot(ax=ax, color="purple", alpha=0.3)

# Measurements & predictions
false_negatives.plot(ax=ax, color="red", alpha=0.6, markersize=26)
false_positives.plot(ax=ax, color="black", alpha=0.6, markersize=26)
true_positives.plot(ax=ax, color="green", alpha=0.8, markersize=26)

# Background
cx.add_basemap(ax=ax, source=cx.providers.Esri.WorldImagery, crs=CRS)

ax.axis("off")

# Create legend
ar = mpatches.Patch(facecolor="silver", edgecolor="blue", label="area")
tp = mpatches.Patch(color="green", label="true positive")
fp = mpatches.Patch(color="black", alpha=0.6, label="false positive")
fn = mpatches.Patch(color="red", alpha=0.6, label="false negative")
pr = mpatches.Patch(color="purple", alpha=0.3, label="predictions (AHN) - trees")
plt.legend(handles=[ar, tp, fp, fn, pr], loc="lower right")

plt.savefig(output_image_analysis, bbox_inches="tight")
plt.show()